<a href="https://colab.research.google.com/github/tincorpai/Recommendation-System/blob/master/Content_Based_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based Recommendation System 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/Shared drives/machinelearning/deep_learning_with_python/recommendation_python')

In [ ]:
#import dataset
data = pd.read_csv("movies_clean.csv")

In [ ]:
data.head()

,Unnamed: 0,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,cast,crew
0,0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Legend Ends,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Lost in our world, found in another.",6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
#Get the shape of the imported dataset
data.shape

(4803, 19)

In [ ]:
#Get the information of the imported dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  release_date          4802 non-null   object 
 11  revenue               4803 non-null   int64  
 12  runtime               4801 non-null   float64
 13  spoken_languages      4803 non-null   object 
 14  tagline               3959 non-null   object 
 15  vote_average         

In [ ]:
#Get the sum of missing values
data.overview.isnull().sum()

3

In [ ]:
data.overview = data.overview.fillna('')

In [ ]:
data.overview.isnull().sum()

0

## Prepare data for Content Based Recommendation System

We need to convert these strings in the overview feature into a vector representation.

>We need to convert our text data into float or integers. In order to do so, we will make use of some NLP technique to convert our text data into some vectors and then pass our vectors into the machine learning model to perform some predictions.

To convert the text data into vector, we will use bag or words to perform some the steps on our text data

   > Step 1:  Lowering the sentences

   > Step 2:  Apply tokenization on our data. This is about splitting our text data into tokens with respect to our text.

   >Step 3:   Historgram: Calculate the frequency of our tokens

   > Step 4:  Perform filteration. Filter words remove less frequent votes or frequency.

   > Step 5:  Create a matrix or document matrix that you pass to your machine learning model for prediction.

**Note** that in NLP, sentences are set up in as rows while words in individual sentences are setup as features to make matrix that contains vectors.
This will now be the final matrix that will be pass to the machine learning model to make some kind of prediction.

TF-IDF will assign priority according to important of words in our text data.

## Intuition Behind TF_IDF 

TF IDF assign importance to all dataset in the text data.'

Steps to performing TF Step:

Step1:   Apply lowercase operation on the dataset.

Step2 :  Compute TF formula: Number 
\begin{align}
        TF = \frac{no of occurences of a word in a document}{no of words on the documents}.
    \end{align}

IDF : It is a document frequency 

You can compute IDF using this formula:

   \begin{align}
        TF = \frac{Total no of docs}{no of doc containing words}.
    \end{align}

Then to get the TF IDF final dataset, multiply `TF*IDF` together in order to get the final matrix to be passsed to the ML algorithm.

Its returns the vector representation of our text data.

In [ ]:
data.overview

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfv=TfidfVectorizer(min_df=3,
                    max_features=None,
                    ngram_range=(1, 3),
                    stop_words='english')

In [ ]:
#fit the tfv to the dataset
tfv_matrix = tfv.fit_transform(data['overview'])

In [ ]:
tfv_matrix.shape

(4803, 9919)

In [ ]:
tfv_matrix[0].T.toarray()

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [ ]:
data_vector = tfv_matrix.toarray()

In [ ]:
#Transform the sparse data to a dataframe 
df2 = pd.DataFrame(data_vector)

## Applying sigmoid kernel on the Dataset

We want to finc the similarity score among the observation in our dataset.

We will use the Cosine similarity to calculate some numerical quantity.
For instance, the similarity score is `0.7` then we may say that there is 0.7 probability that the two observatons are similar.

Apply sigmoid on top of vectors that we have because sigmoid typically convert values from the range of 0 to 1.


In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

In [ ]:
sig =sigmoid_kernel(tfv_matrix, tfv_matrix)

In [ ]:
sig

array([[0.76163649, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76163649, 0.76159416, ..., 0.76159519, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76163649, ..., 0.76159484, 0.76159416,
        0.76159416],
       ...,
       [0.76159416, 0.76159519, 0.76159484, ..., 0.76163649, 0.76159488,
        0.76159447],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159488, 0.76163649,
        0.76159467],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159447, 0.76159467,
        0.76163649]])

In [ ]:
index = data.index
index

RangeIndex(start=0, stop=4803, step=1)

In [ ]:
title = data['original_title']
title[4802]

'My Date with Drew'

In [ ]:
# get the index of all title of the movies
indices = pd.Series(data.index, index = data['original_title'])

In [ ]:
list(sig[indices['Avatar']])

[0.7616364930962501,
 0.7615941559557649,
 0.7615941559557649,
 0.761595125164868,
 0.7615941559557649,
 0.761595309447695,
 0.7615941559557649,
 0.761595908255829,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615950846887262,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.76159572705,
 0.7615962614863476,
 0.7615941559557649,
 0.7615952950243241,
 0.7615941559557649,
 0.7615968611459101,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.761597988130195,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615941559557649,
 0.7615951605621958,
 0.7615941559557649,
 0.7615968356260668,
 0.7615941559557649,
 0.7615956092492897,
 

In [ ]:
len(list(sig[indices['Avatar']]))

4803

In [ ]:
list(enumerate(list(sig[indices['Avatar']])))

[(0, 0.7616364930962501),
 (1, 0.7615941559557649),
 (2, 0.7615941559557649),
 (3, 0.761595125164868),
 (4, 0.7615941559557649),
 (5, 0.761595309447695),
 (6, 0.7615941559557649),
 (7, 0.761595908255829),
 (8, 0.7615941559557649),
 (9, 0.7615941559557649),
 (10, 0.7615941559557649),
 (11, 0.7615950846887262),
 (12, 0.7615941559557649),
 (13, 0.7615941559557649),
 (14, 0.7615941559557649),
 (15, 0.7615941559557649),
 (16, 0.7615941559557649),
 (17, 0.7615941559557649),
 (18, 0.7615941559557649),
 (19, 0.7615941559557649),
 (20, 0.7615941559557649),
 (21, 0.7615941559557649),
 (22, 0.7615941559557649),
 (23, 0.7615941559557649),
 (24, 0.7615941559557649),
 (25, 0.7615941559557649),
 (26, 0.76159572705),
 (27, 0.7615962614863476),
 (28, 0.7615941559557649),
 (29, 0.7615952950243241),
 (30, 0.7615941559557649),
 (31, 0.7615968611459101),
 (32, 0.7615941559557649),
 (33, 0.7615941559557649),
 (34, 0.7615941559557649),
 (35, 0.7615941559557649),
 (36, 0.761597988130195),
 (37, 0.761594155955

In [ ]:
#Sort all these coeeficient 
sigma_scores = sorted(list(enumerate(list(sig[indices['Avatar']]))), key=lambda x:x[1], reverse = True)

In [ ]:
sigma_scores

[(0, 0.7616364930962501),
 (1341, 0.7616030155858681),
 (634, 0.7616028561141562),
 (3604, 0.761601930611584),
 (2130, 0.7616015339622925),
 (775, 0.7616011086528327),
 (529, 0.7615996114069044),
 (151, 0.7615991171152051),
 (311, 0.7615990624497703),
 (847, 0.7615987706430225),
 (570, 0.7615986450599548),
 (942, 0.7615984376900236),
 (36, 0.761597988130195),
 (1610, 0.7615979793934843),
 (3070, 0.7615978406764746),
 (1033, 0.7615978182403835),
 (2628, 0.7615977834088159),
 (1784, 0.7615977150705628),
 (2578, 0.7615976778191441),
 (150, 0.7615976453752453),
 (3724, 0.7615975951237102),
 (1013, 0.761597590729192),
 (4211, 0.7615975631290406),
 (1213, 0.7615975380289366),
 (1345, 0.7615974549075267),
 (312, 0.7615974086679764),
 (4039, 0.7615973645677722),
 (2967, 0.7615973512232982),
 (614, 0.7615972949789032),
 (281, 0.7615972537743877),
 (174, 0.7615972462403858),
 (3493, 0.7615971922075142),
 (3624, 0.7615971821325882),
 (972, 0.7615971791001622),
 (1274, 0.7615971587672579),
 (1959,

In [ ]:
# We need to store all these indexes within a list 
sigma_scores[1:11]

[(1341, 0.7616030155858681),
 (634, 0.7616028561141562),
 (3604, 0.761601930611584),
 (2130, 0.7616015339622925),
 (775, 0.7616011086528327),
 (529, 0.7615996114069044),
 (151, 0.7615991171152051),
 (311, 0.7615990624497703),
 (847, 0.7615987706430225),
 (570, 0.7615986450599548)]

In [ ]:
ind = [index[0] for index in sigma_scores[1:11]]
ind

[1341, 634, 3604, 2130, 775, 529, 151, 311, 847, 570]

In [ ]:
#then we get the index location of these index
data['original_title'].iloc[ind]

1341                Obitaemyy Ostrov
634                       The Matrix
3604                       Apollo 18
2130                    The American
775                        Supernova
529                 Tears of the Sun
151                          Beowulf
311     The Adventures of Pluto Nash
847                         Semi-Pro
570                           Ransom
Name: original_title, dtype: object

Which means these are all the movies that are recommended to us.

In [ ]:
def give_rec(title, model):
  idx = indices[title]

  model_scores = list(enumerate(list(model[indices[idx]])))

  model_scores = sorted(model_scores, key=lambda x:x[1], reverse = True)

  model_scores = model_scores[1:11]

  movies_indices = [index[0] for index in model_scores]

  return data['original_title'].iloc[movies_indices]









In [ ]:
give_rec('Avatar', sig)


1341                Obitaemyy Ostrov
634                       The Matrix
3604                       Apollo 18
2130                    The American
775                        Supernova
529                 Tears of the Sun
151                          Beowulf
311     The Adventures of Pluto Nash
847                         Semi-Pro
570                           Ransom
Name: original_title, dtype: object

In [ ]:
data.original_title

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: original_title, Length: 4803, dtype: object

## Understand how to improve the model from business perspectives

In [ ]:
from ast import literal_eval

In [ ]:
data.columns

Index(['Unnamed: 0', 'budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [ ]:
literal_eval(data['crew'][0])

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [ ]:
features = ['cast', 'crew', 'keywords', 'genres']

In [ ]:
for feature in features:
  data[feature] = data[feature].apply(literal_eval)

In [ ]:
data[features]

,cast,crew,keywords,genres
0,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
1,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
3,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
4,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
...,...,...,...,...
4798,"[{'cast_id': 1, 'character': 'El Mariachi', 'c...","[{'credit_id': '52fe44eec3a36847f80b280b', 'de...","[{'id': 5616, 'name': 'united states–mexico ba...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
4799,"[{'cast_id': 1, 'character': 'Buzzy', 'credit_...","[{'credit_id': '52fe487dc3a368484e0fb013', 'de...",[],"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '..."
4800,"[{'cast_id': 8, 'character': 'Oliver O’Toole',...","[{'credit_id': '52fe4df3c3a36847f8275ecf', 'de...","[{'id': 248, 'name': 'date'}, {'id': 699, 'nam...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4801,"[{'cast_id': 3, 'character': 'Sam', 'credit_id...","[{'credit_id': '52fe4ad9c3a368484e16a36b', 'de...",[],[]


## Let's perform feature extraction.

Feature extraction is part of our domain knowledge

In [ ]:
def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name'].strip()

  else:
    return np.nan


In [ ]:
data['director'] = data['crew'].apply(get_director)
data.director

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4          Andrew Stanton
              ...        
4798     Robert Rodriguez
4799         Edward Burns
4800          Scott Smith
4801          Daniel Hsia
4802     Brian Herzlinger
Name: director, Length: 4803, dtype: object

In [ ]:
#Get the shape of the director dataset
data['director'].shape

(4803,)

In [ ]:
data.director.isnull().sum()

30

In [ ]:
data['cast'][0]

[{'cast_id': 242,
  'character': 'Jake Sully',
  'credit_id': '5602a8a7c3a3685532001c9a',
  'gender': 2,
  'id': 65731,
  'name': 'Sam Worthington',
  'order': 0},
 {'cast_id': 3,
  'character': 'Neytiri',
  'credit_id': '52fe48009251416c750ac9cb',
  'gender': 1,
  'id': 8691,
  'name': 'Zoe Saldana',
  'order': 1},
 {'cast_id': 25,
  'character': 'Dr. Grace Augustine',
  'credit_id': '52fe48009251416c750aca39',
  'gender': 1,
  'id': 10205,
  'name': 'Sigourney Weaver',
  'order': 2},
 {'cast_id': 4,
  'character': 'Col. Quaritch',
  'credit_id': '52fe48009251416c750ac9cf',
  'gender': 2,
  'id': 32747,
  'name': 'Stephen Lang',
  'order': 3},
 {'cast_id': 5,
  'character': 'Trudy Chacon',
  'credit_id': '52fe48009251416c750ac9d3',
  'gender': 1,
  'id': 17647,
  'name': 'Michelle Rodriguez',
  'order': 4},
 {'cast_id': 8,
  'character': 'Selfridge',
  'credit_id': '52fe48009251416c750ac9e1',
  'gender': 2,
  'id': 1771,
  'name': 'Giovanni Ribisi',
  'order': 5},
 {'cast_id': 7,
  'c

In [ ]:
type(data['cast'][0])

list

In [ ]:
def get_list(x):
  names = [i['name'].strip().lower() for i in x if type(x) == list]

  if len(names)>3:
    return names[0:3]
  else:
    return names



In [ ]:
get_list(data['crew'][0])

['stephen e. rivkin', 'rick carter', 'christopher boyes']

In [ ]:
#apply the function on the four features we have 
for feature in features:
  data[feature] = data[feature].apply(get_list)

In [ ]:
data[features]

,cast,crew,keywords,genres
0,"[sam worthington, zoe saldana, sigourney weaver]","[stephen e. rivkin, rick carter, christopher b...","[culture clash, future, space war]","[action, adventure, fantasy]"
1,"[johnny depp, orlando bloom, keira knightley]","[dariusz wolski, gore verbinski, jerry bruckhe...","[ocean, drug abuse, exotic island]","[adventure, fantasy, action]"
2,"[daniel craig, christoph waltz, léa seydoux]","[thomas newman, sam mendes, anna pinnock]","[spy, based on novel, secret agent]","[action, adventure, crime]"
3,"[christian bale, michael caine, gary oldman]","[hans zimmer, charles roven, christopher nolan]","[dc comics, crime fighter, terrorist]","[action, crime, drama]"
4,"[taylor kitsch, lynn collins, samantha morton]","[andrew stanton, andrew stanton, john lasseter]","[based on novel, mars, medallion]","[action, adventure, science fiction]"
...,...,...,...,...
4798,"[carlos gallardo, jaime de hoyos, peter marqua...","[robert rodriguez, robert rodriguez, robert ro...","[united states–mexico barrier, legs, arms]","[action, crime, thriller]"
4799,"[edward burns, kerry bishé, marsha dietlein]","[edward burns, edward burns, edward burns]",[],"[comedy, romance]"
4800,"[eric mabius, kristin booth, crystal lowe]","[carla hetland, harvey kahn, adam sliwinski]","[date, love at first sight, narration]","[comedy, drama, romance]"
4801,"[daniel henney, eliza coupe, bill paxton]","[daniel hsia, daniel hsia]",[],[]


In [ ]:
data.isnull().sum()

Unnamed: 0                0
budget                    0
genres                    0
id                        0
keywords                  0
original_language         0
original_title            0
overview                  0
popularity                0
production_companies      0
release_date              1
revenue                   0
runtime                   2
spoken_languages          0
tagline                 844
vote_average              0
vote_count                0
cast                      0
crew                      0
director                 30
dtype: int64

In [ ]:
#drop all missing values
data.dropna(subset=['director'], inplace=True)

In [ ]:
data.isnull().sum()

Unnamed: 0                0
budget                    0
genres                    0
id                        0
keywords                  0
original_language         0
original_title            0
overview                  0
popularity                0
production_companies      0
release_date              0
revenue                   0
runtime                   2
spoken_languages          0
tagline                 822
vote_average              0
vote_count                0
cast                      0
crew                      0
director                  0
dtype: int64

## What is Meta Data and how to create a meta-data

A meta data is a collection of cast, crew, keywords, genres and directors.

In [ ]:
data['cast'][0]

['sam worthington', 'zoe saldana', 'sigourney weaver']

In [ ]:
' '.join(data['cast'][0])

'sam worthington zoe saldana sigourney weaver'

In [ ]:
def create_feature(row):
  return ' '.join(row['keywords']) + ' ' + ' '.join(row['cast']) + ' ' + row['director'] + ' '.join(row['genres'])

In [ ]:
data['important'] = data.apply(create_feature, axis=1)

In [ ]:
data.head(2)

,Unnamed: 0,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,cast,crew,director,important
0,0,237000000,"[action, adventure, fantasy]",19995,"[culture clash, future, space war]",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800,"[sam worthington, zoe saldana, sigourney weaver]","[stephen e. rivkin, rick carter, christopher b...",James Cameron,culture clash future space war sam worthington...
1,1,300000000,"[adventure, fantasy, action]",285,"[ocean, drug abuse, exotic island]",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",6.9,4500,"[johnny depp, orlando bloom, keira knightley]","[dariusz wolski, gore verbinski, jerry bruckhe...",Gore Verbinski,ocean drug abuse exotic island johnny depp orl...


In [ ]:
#we do feature extraction
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Create a model for vectorization
count = CountVectorizer(stop_words='english')

In [ ]:
count_matrix = count.fit_transform(data['important'])
count_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Recommend a movie 


In [ ]:
from sklearn.metrics.pairwise import  cosine_similarity

In [ ]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2[0]

array([1.        , 0.06666667, 0.13333333, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
give_rec('Tangled', cosine_sim2)

143                                         Pan
1589                             Racing Stripes
2432                            American Dreamz
2351                           Love the Coopers
578     Alvin and the Chipmunks: The Squeakquel
1511          Dreamer: Inspired By a True Story
1108                                  Pinocchio
1391                             License to Wed
1788                                    Red Eye
1857                            Rugrats Go Wild
Name: original_title, dtype: object

In [ ]:
give_rec('The Godfather', cosine_sim2)

2731     The Godfather: Part II
1525             Apocalypse Now
867     The Godfather: Part III
512                      Wanted
1209              The Rainmaker
4147           Small Apartments
4124         This Thing of Ours
2649          The Son of No One
3765         Let's Go to Prison
613                   The Score
Name: original_title, dtype: object

In [ ]:
give_rec('The Godfather', sig)

2731                             The Godfather: Part II
1873                                         Blood Ties
3730                                              Cargo
867                             The Godfather: Part III
3623                                               Made
3963                                 They Came Together
2464                             The Master of Disguise
2973    For Greater Glory - The True Story of Cristiada
2389                                        Renaissance
1475                                  Playing for Keeps
Name: original_title, dtype: object

## To improve on the model, consider improving on the following:

*  Production

*  Actor

*  Director 